In [1]:
import pickle
import awkward as ak
import numpy as np
import vector
import json

In [2]:
mu=60
num_events=10
pt_cut=1.8 #GeV

In [3]:
with open("WS_Preprocessed_ttbar_semiLep_mu"+str(mu)+"_minJetpT25/data_combined.pkl","rb") as f:
    data = pickle.load(f)

In [4]:
trks = data["trks"]
isHS = trks[:,:,-1]==-1

In [5]:
trk_vect = vector.Array(ak.zip({"pt": trks[:,:,0],"eta": trks[:,:,1],"phi": trks[:,:,2],"m": ak.zeros_like(trks[:,:,0])}))

In [6]:
# Initialize the structure
data = {
    "Test": {
        "Tracks": {
            "TestTracks": []
        },
        "event number": 999,
        "run number": 999
    }
}

In [7]:
def generate_curved_path(start_point, end_point, radius, num_points=10):
    """
    Generate a curved path between two points based on magnetic field curvature.
    
    Parameters:
    - start_point: [x, y, z] starting position
    - end_point: [x, y, z] ending position  
    - radius: curvature radius (smaller = more curved)
    - num_points: number of points to generate along the curve
    
    Returns:
    - List of [x, y, z] points along the curved path
    """
    
    # Vector from start to end
    dx = end_point[0] - start_point[0]
    dy = end_point[1] - start_point[1]
    dz = end_point[2] - start_point[2]
    
    # Length of the straight line
    straight_length = np.sqrt(dx**2 + dy**2 + dz**2)
    
    # If radius is very large or zero, return straight line
    if radius <= 0 or radius > straight_length * 10:
        # Return straight line points
        path = []
        for i in range(num_points):
            t = i / (num_points - 1) if num_points > 1 else 0
            path.append([
                start_point[0] + t * dx,
                start_point[1] + t * dy,
                start_point[2] + t * dz
            ])
        return path
    
    # Calculate the curvature (how much the path deviates from straight line)
    # This is a simplified model - in reality, magnetic field direction matters
    # For simplicity, we'll assume the curve is in the plane perpendicular to momentum
    
    # Create a simple circular arc approximation
    path = []
    
    # For a circular path, we need to determine the center and angle
    # This is a simplified approach - the actual physics is more complex
    
    # Simple approach: interpolate between start and end, with some curvature
    for i in range(num_points):
        t = i / (num_points - 1) if num_points > 1 else 0
        
        # Base point (straight line)
        base_x = start_point[0] + t * dx
        base_y = start_point[1] + t * dy
        base_z = start_point[2] + t * dz
        
        # Add curvature effect - more curvature for smaller momentum
        # This is a simplified model
        curvature_factor = 1.0 / (radius + 1)  # +1 to avoid division by zero
        
        # Simple curvature in a plane (this would need to be more sophisticated)
        # For now, we'll add some displacement perpendicular to the path
        if num_points > 2:
            # Add some oscillation to make it look curved
            oscillation = curvature_factor * 0.1 * (t - 0.5) * straight_length
            # Simple displacement (would need to be more physics-based)
            path.append([base_x, base_y, base_z + oscillation])
        else:
            path.append([base_x, base_y, base_z])
    
    return path

def generate_realistic_curved_path(start_point, end_point, momentum_norm, num_points=10):
    """
    Generate a more realistic curved path based on momentum magnitude.
    
    Parameters:
    - start_point: [x, y, z] starting position
    - end_point: [x, y, z] ending position  
    - momentum_norm: magnitude of momentum (higher = less curved)
    - num_points: number of points to generate along the curve
    
    Returns:
    - List of [x, y, z] points along the curved path
    """
    
    # Calculate curvature radius (inverse proportional to momentum)
    # You can adjust this formula based on your physics requirements
    if momentum_norm > 0:
        # Higher momentum = larger radius = less curved
        curvature_radius = 10 / momentum_norm  # Adjust this scaling factor
    else:
        curvature_radius = 1000000  # Default for zero momentum
    
    # Cap the radius to prevent extreme curves
    curvature_radius = max(100, min(1000000, curvature_radius))
    
    # Generate the curved path
    return generate_curved_path(start_point, end_point, curvature_radius, num_points)

In [8]:
for i in range(num_events):
    num_tracks = len(trk_vect[i])
    for j in range(num_tracks):
        if trk_vect[i][j].pt<pt_cut:
            continue
        px = trk_vect[i][j].px
        py = trk_vect[i][j].py
        pz = trk_vect[i][j].pz
    
        norm = np.sqrt(px**2+py**2+pz**2)
        scale = 10000
    
        px = px*scale/norm
        py = py*scale/norm
        pz = pz*scale/norm
        
        color = isHS[i][j]
    
        # Convert boolean to hex color
        hex_color = "0000FF" if color else "FF0000"
        
        # Create start and end points
        start_point = [0.0, 0.0, 0.0]
        end_point = [px, py, pz]
        
        # Generate curved path with multiple points
        # You can adjust num_points as needed
        curved_path = generate_realistic_curved_path(start_point, end_point, norm, num_points=15)
        
        # Create the track entry with curved path
        track_entry = {
            "color": hex_color,
            "pos": curved_path  # This now contains multiple points along the curved path
        }
            
        # Add to the TestTracks list
        data["Test"]["Tracks"]["TestTracks"].append(track_entry)
        
        # Save to JSON file
    with open('phoenix/event_'+str(i)+'_mu'+str(mu)+'.json', 'w') as f:
        json.dump(data, f, indent=2)